In [11]:
import json
import requests
from datetime import datetime
import pytz
from postgres_driver import PostgresDatabaseDriver

In [12]:
response = requests.get('http://site.api.espn.com/apis/site/v2/sports/football/nfl/summary?event=401671789')
json_data = response.json()

In [13]:
player_stats = json_data['boxscore']['players']

In [14]:
def insert_player_stats_data(driver, stats, filename, etl_ts):
    try:
        insert_statement = """
            INSERT INTO landing.player_stats (stats_json, filename, etl_ts)
            VALUES (%s, %s, %s)
        """
        for stat in stats:
            stat_json = json.dumps(stat)
            driver.execute(insert_statement, (stat_json, filename, etl_ts))
        driver.commit()
    except Exception as e:
        print("Error inserting events:", e)
        driver.conn.rollback()

In [15]:
driver = PostgresDatabaseDriver()

In [16]:
if player_stats:
    eastern_tz = pytz.timezone('US/Eastern')
    ts = datetime.now(eastern_tz)
    etl_ts = ts.isoformat()
    insert_player_stats_data(driver, player_stats, 'games/401671789.json', etl_ts)
    print(f"Player stats in games/401671789.json uploaded successfully")
else:
    print(f"No teams found in games/401671789.json")

Player stats in games/401671789.json uploaded successfully
